# Step 0: Import and Reading Data

In [4]:
# Importando bibliotecas e dataset
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

plt.style.use('ggplot')
# pd.set_option('max_columns', 200)


In [5]:
df = pd.read_csv('defeitos_dados.csv')

In [6]:
df.head()

,id_defeito,id_produto,tipo_defeito,data_defeito,local_defeito,severidade,metodo_inspecao,custo_reparo
0,1,15,Structural,06/06/2024,Component,Minor,Visual Inspection,245.47
1,2,6,Functional,26/04/2024,Component,Minor,Visual Inspection,26.87
2,3,84,Structural,15/02/2024,Internal,Minor,Automated Testing,835.81
3,4,10,Functional,28/03/2024,Internal,Critical,Automated Testing,444.47
4,5,14,Cosmetic,26/04/2024,Component,Minor,Manual Testing,823.64


# Step 1: Data Understanding

* Dataframe shape
* head and tail
* dtypes
* describe


In [7]:
# Exibe a quantidade de linhas e colunas
df.shape

(1000, 8)

In [8]:
# Exibe as 5 primeiras colunas
df.head()

,id_defeito,id_produto,tipo_defeito,data_defeito,local_defeito,severidade,metodo_inspecao,custo_reparo
0,1,15,Structural,06/06/2024,Component,Minor,Visual Inspection,245.47
1,2,6,Functional,26/04/2024,Component,Minor,Visual Inspection,26.87
2,3,84,Structural,15/02/2024,Internal,Minor,Automated Testing,835.81
3,4,10,Functional,28/03/2024,Internal,Critical,Automated Testing,444.47
4,5,14,Cosmetic,26/04/2024,Component,Minor,Manual Testing,823.64


In [9]:
# Exibe o nome das colunas
df.columns

Index(['id_defeito', 'id_produto', 'tipo_defeito', 'data_defeito',
       'local_defeito', 'severidade', 'metodo_inspecao', 'custo_reparo'],
      dtype='object')

In [10]:
# Seleciona apenas as colunas desejadas e cria uma cópia independente do DataFrame

df_filt = df[[
    'data_defeito',
    'local_defeito',
    'severidade',
    'metodo_inspecao',
    'custo_reparo'
]].copy()


In [11]:
# Exemplo de remoção de colunas
df.drop(['local_defeito'], axis=1)

,id_defeito,id_produto,tipo_defeito,data_defeito,severidade,metodo_inspecao,custo_reparo
0,1,15,Structural,06/06/2024,Minor,Visual Inspection,245.47
1,2,6,Functional,26/04/2024,Minor,Visual Inspection,26.87
2,3,84,Structural,15/02/2024,Minor,Automated Testing,835.81
3,4,10,Functional,28/03/2024,Critical,Automated Testing,444.47
4,5,14,Cosmetic,26/04/2024,Minor,Manual Testing,823.64
...,...,...,...,...,...,...,...
995,996,25,Structural,01/03/2024,Minor,Automated Testing,813.14
996,997,23,Functional,21/03/2024,Moderate,Automated Testing,944.07
997,998,17,Structural,16/01/2024,Minor,Automated Testing,401.12
998,999,96,Cosmetic,21/06/2024,Moderate,Manual Testing,775.63


In [12]:
#Exibe as colunas e seu tipo de dados
df.dtypes

id_defeito           int64
id_produto           int64
tipo_defeito        object
data_defeito        object
local_defeito       object
severidade          object
metodo_inspecao     object
custo_reparo       float64
dtype: object

In [13]:
df.describe()

,id_defeito,id_produto,custo_reparo
count,1000.000000,1000.000000,1000.000000
mean,500.500000,50.837000,507.627150
std,288.819436,29.480935,289.623615
min,1.000000,1.000000,10.220000
25%,250.750000,26.000000,270.902500
50%,500.500000,51.000000,506.430000
75%,750.250000,77.000000,759.065000
max,1000.000000,100.000000,999.640000


In [14]:
# Verifica se existem valores nulos nas colunas
df_filt.isna().sum()

data_defeito       0
local_defeito      0
severidade         0
metodo_inspecao    0
custo_reparo       0
dtype: int64

In [15]:
# Retorna todas as linhas duplicadas, exceto a primeira ocorrência de cada grupo de duplicatas
df_filt.loc[df_filt.duplicated()]

,data_defeito,local_defeito,severidade,metodo_inspecao,custo_reparo
